In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot as plt
from pandas import Series
from datetime import datetime
from ipywidgets import IntProgress
from multiprocessing import Pool, cpu_count
import pickle
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from fbprophet import Prophet
import os
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

### GLOBAL VARIABLES

In [ ]:
INPUT_PATH = '../../data/train_test'
OUTPUT_PATH = '../../data/train_test'
OUTPUT_NAME = 'submission_013'
DAYS_PRED = 28
METRIC = 'rmse'
cfg_prophet = dict()

### FUNCTIONS

In [ ]:
# score a model, return None on failure
def run_prophet(data, period=DAYS_PRED):
        
    
    # holiday
    calendar = pd.read_csv(f'../../data/raw/calendar.csv')
    holidays = calendar[calendar.event_name_1.isnull()!=True][['date','event_name_1']]
    holidays.columns = ['ds','holiday']
    
    item_id = data.item_id.unique()[0]
    store_id = data.store_id.unique()[0]
    
    cfg_prophet = dict()
    cfg_prophet['yearly_seasonality'] = False
    cfg_prophet['daily_seasonality'] = False
    cfg_prophet['weekly_seasonality'] = False
    cfg_prophet['seasonality_prior_scale'] = 0.1
    cfg_prophet['holidays'] = holidays
   
    m = Prophet(**cfg_prophet).add_seasonality(
        name='monthly', period=30.5, fourier_order=12).add_seasonality(
        name='weekly', period=7, fourier_order=10).add_seasonality(
        name='yearly', period=365.25, fourier_order=12).add_seasonality(
        name='quarterly', period=365.25/4, fourier_order=5, prior_scale=0.1)
    
    

    # show all warnings and fail on exception if debugging
    with catch_warnings():
        filterwarnings("ignore")

        m.fit(data[['ds','y']])
        future = m.make_future_dataframe(periods=period,include_history=True)

        forecast = m.predict(future)
        
        forecast['item_id'] = item_id
        forecast['store_id'] = store_id
    

    return forecast

### LOAD DATASET

In [ ]:
X_train_vector = pickle.load(open( f'{INPUT_PATH}/X_train.pkl', "rb" )) # It loads a vector with the folds

In [ ]:
Y_train_vector = pickle.load(open( f'{INPUT_PATH}/Y_train.pkl', "rb" ))

In [ ]:
X_val_vector = pickle.load(open( f'{INPUT_PATH}/X_val.pkl', "rb" )) # It loads a vector with the folds

In [ ]:
Y_val_vector = pickle.load(open( f'{INPUT_PATH}/Y_val.pkl', "rb" )) # It loads a vector with the folds

In [ ]:
calendar = pd.read_csv(f'../../data/raw/calendar.csv')

### TRAIN MODEL AND MAKE FEATURES

In [ ]:
features = [
 'trend',
  'monthly',
 'quarterly',
 'weekly',
 'yearly',
 'multiplicative_terms',
]



for fold in range(len(X_train_vector)):
    print(f"\n----- Fold: ({fold + 1} / {len(X_train_vector)}) -----\n")
    train = X_train_vector[fold].join(Y_train_vector[fold])
    val = X_val_vector[fold]
    
    # sorting
    train.sort_values(['item_id','store_id','date'], inplace=True, ascending=True)
    val.sort_values(['item_id','store_id','date'], inplace=True, ascending=True)
    
    
    forecast_df = pd.DataFrame()
    STEP=547
    init = 0
    train_vector = []

    for n in range(1,30490+1):
        end = STEP*n
        ts = train.iloc[init:end, [0,3,38,39]]
        ts.columns = ['item_id', 'store_id', 'ds','y']
        
        train_vector.append(ts)
        init+=STEP
        
        
        
    p = Pool(cpu_count())
    predictions = list(tqdm_notebook(p.imap(run_prophet, train_vector), total=len(train_vector)))
    p.close()
    p.join()
    
    for pred in predictions:
        forecast_df = forecast_df.append(pred[features + ['item_id','store_id','ds']])
    
   
    
    # Unimos dataset train y validation con los valores de forecast y reeamplazamos en el vector principal
    train = pd.merge(train, forecast_df[features + ['ds']], how='inner',
                     left_on=['item_id','store_id','date'], right_on=['item_id','store_id','ds'])
    
    val = pd.merge(val, forecast_df[features + ['ds']], how='inner',
                     left_on=['item_id','store_id','date'], right_on=['item_id','store_id','ds'])
        
    train.drop(['ds','demand_smoothed'],axis=1, inplace=True)
    val.drop(['ds'],axis=1, inplace=True)
    
    X_train_vector[fold] = train
    X_val_vector[fold] = val
